In [1]:
import os

In [2]:
%pwd

'e:\\Risk-Analysis\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'e:\\Risk-Analysis'

In [6]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class DataTransformationConfig:
    transformation_dir:Path
    preprocessor_dir :Path
    


In [7]:
from src.Risk_Analysis.constants import config_file
from src.Risk_Analysis.constants import params_file
from src.Risk_Analysis.utils.common import real_yaml,create_directories

In [9]:
class Configuration:
    def __init__(self,config=config_file,params=params_file):
        self.config=real_yaml(config)
        self.params=real_yaml(params)
        create_directories([self.config.Artifacts])

    def get_transformations_config(self)->DataTransformationConfig:
        data_trans=self.config.Data_Transformation

        create_directories([data_trans.transformation_dir])

        data_transformation=DataTransformationConfig(
            transformation_dir=data_trans.transformation_dir,
            preprocessor_dir=data_trans.preprocessor_dir
        )

        return data_transformation

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np
from src.Risk_Analysis.utils.common import save_oject

In [11]:
class DataTransformation:
    def __init__(self,config):
        self.config=config

    def get_data_transformation(self):

        cat_features=['Days_took_Disbursement']
        numerical_features=['Term', 'NoEmp', 'NewExist', 'CreateJob', 'RetainedJob', 'UrbanRural',
                            'RevLineCr', 'LowDoc', 'DisbursementGross', 'GrAppv', 'SBA_Appv']



        cat_pip=Pipeline(steps=[
            ('imputer',SimpleImputer(strategy='most_frequent'))
            ])
        num_pip=Pipeline(steps=[
            ('scaler',StandardScaler()),
            ('imputer',SimpleImputer(strategy='median'))
            ])
        


        
        preprocessor=ColumnTransformer([
            ('num_pip',num_pip,numerical_features),
            ('cat_pip',cat_pip,cat_features)
            ])
        return preprocessor
    
    def data_transformation_initiated(self):

        train_set=pd.read_csv('artifacts\\data_ingestion\\train.csv')

        test_set=pd.read_csv('artifacts\\data_ingestion\\test.csv')

        target_column='MIS_Status'

        x_train,x_test,y_train,y_test=(train_set.drop(target_column,axis=1),
                                      test_set.drop(target_column,axis=1),
                                      train_set[target_column],
                                      test_set[target_column])
        preprocessor=self.get_data_transformation()

        x_train=preprocessor.fit_transform(x_train)
        x_test=preprocessor.transform(x_test)

        save_oject(self.config.preprocessor_dir,preprocessor)

        return x_train,x_test,y_train,y_test


In [12]:
try:
    transformation_config=Configuration()
    config=transformation_config.get_transformations_config()

    data_transformation=DataTransformation(config)
    x_train,x_test,y_test,y_train=data_transformation.data_transformation_initiated()

except Exception as e:
    raise e

[2023-09-24 13:31:24,819 - Risk Analysis - INFO - directory is created ]
[2023-09-24 13:31:24,819 - Risk Analysis - INFO - directory is created ]
